In [1]:
import requests
import bs4
from bs4 import BeautifulSoup
from urllib.parse import urljoin
import pandas as pd

In [19]:
url_inicial = 'https://produccioncientificaluz.org/index.php/investigacion/issue/archive/2'
url_root = 'https://produccioncientificaluz.org/index.php/investigacion/issue/archive/2'
r=requests.get(url_inicial)

In [20]:
soup = BeautifulSoup(r.text, 'html.parser')

In [21]:
box = soup.find('ul', class_='issues_archive')
volumen=box.findAll('div', class_='obj_issue_summary')
vol = [x.find('a').get('href')for x in volumen]
vol=[urljoin(url_root,i) for i in vol]
vol

['https://produccioncientificaluz.org/index.php/investigacion/issue/view/3028',
 'https://produccioncientificaluz.org/index.php/investigacion/issue/view/3027',
 'https://produccioncientificaluz.org/index.php/investigacion/issue/view/3026']

In [22]:
vol2 =[]
for i in vol: 
    url_inicial1=i 
    r1 = requests.get(url_inicial1)
    soup1 = BeautifulSoup(r1.text, 'html.parser')
    box1 = soup1.find('div', class_='sections') 
    volumen1=box1.findAll('div', class_='obj_article_summary')
    vol1 = [x.find('a').get('href')for x in volumen1]
    #box1.text.split("RESUMEN")[1].split("REFERENCIAS")[0].strip()
    vol2+=vol1

In [23]:
vol2

['https://produccioncientificaluz.org/index.php/investigacion/article/view/27632',
 'https://produccioncientificaluz.org/index.php/investigacion/article/view/27633',
 'https://produccioncientificaluz.org/index.php/investigacion/article/view/27634',
 'https://produccioncientificaluz.org/index.php/investigacion/article/view/27635',
 'https://produccioncientificaluz.org/index.php/investigacion/article/view/27636',
 'https://produccioncientificaluz.org/index.php/investigacion/article/view/27637',
 'https://produccioncientificaluz.org/index.php/investigacion/article/view/27627',
 'https://produccioncientificaluz.org/index.php/investigacion/article/view/27628',
 'https://produccioncientificaluz.org/index.php/investigacion/article/view/27629',
 'https://produccioncientificaluz.org/index.php/investigacion/article/view/27630',
 'https://produccioncientificaluz.org/index.php/investigacion/article/view/27631',
 'https://produccioncientificaluz.org/index.php/investigacion/article/view/27621',
 'ht

In [24]:
def get_url_items(sopa,url):
    box1 = soup1.find('div', class_='sections') 
    volumen1=box1.findAll('div', class_='obj_article_summary')
    vol1 = [x.find('a').get('href')for x in volumen1]
    return vol2

In [25]:
links_items=[]
i=0
while i<1:
    i+=1
    print(f'Estoy en la pagina {url_inicial}')
    r_pag=requests.get(url_inicial)
    s_p=BeautifulSoup(r_pag.text,'html.parser')
    links=get_url_items(s_p, url_inicial)
    links_items.append(links) 
    next_a=s_p.select('li.next > a')
    if not next_a or not next_a[0].get('href'):
        break

Estoy en la pagina https://produccioncientificaluz.org/index.php/investigacion/issue/archive/2


In [26]:
list_scraper=[]
for i in links_items:
    for j in i:
        list_scraper.append(j)
len(list_scraper)

17

In [27]:
uno=list_scraper[0]
r_item=requests.get(uno)
s_item=BeautifulSoup(r_item.text, 'html.parser')

In [28]:
#funcion para iniciar el escraper de cada libro
def scraper_book(url):
    content_book={}
    r=requests.get(url)
    tire='Revista de Investigación Clínica'
    a='III. Medicina y Ciencias de la Salud'
    tem='Investigación Clínica'
    s_item=BeautifulSoup(r.text,'html.parser')
 #titulo de revista
    try:
        titulo=tire
        content_book['Titulo Revista']=tire
    except AttributeError:
        content_book['Titulo Revista']=None
    #area
    try:
        area=a
        content_book['Area']=a
    except AttributeError:
        content_book['Area']=None
    #tematica
    try:
        tema=tem
        content_book['Tematica']=tem
    except AttributeError:
        content_book['Tematica']=None
    #titulo articulo
    try:
        titu=s_item.find('h1', class_='page_title').get_text(strip=True)
        content_book['Titulo Articulo']=titu
    except AttributeError:
        content_book['Titulo Articulo']=None
    #resumen
    try:
        resu=s_item.find('div', class_='item abstract').get_text(strip=True)
        content_book['Resumen']=resu.replace('Resumen','')
    except AttributeError:
        content_book['Resumen']=None
    #abstract
    try:
        abst=s_item.find('div', class_='item cover_image').get_text(strip=True)
        content_book['Abstract']=abst
    except AttributeError:
        content_book['Abstract']=None
    #Link incial
    try:
        linkart=url
        content_book['Link Articulo']=linkart
    except AttributeError:
        content_book['Link Articulo']=None
    #link articulo
    try:
        link=s_item.find('a', class_='obj_galley_link pdf').get('href')
        content_book['Link PDF']=urljoin(url_root, link)
    except AttributeError:
        content_book['Link PDF']=None
    return content_book

In [29]:
list_scraper=list_scraper[0:17]
datos_book=[]
for idx, i in enumerate(list_scraper):
    print(f'estas escrapeando la pag {idx}')
    datos_book.append(scraper_book(i))

estas escrapeando la pag 0
estas escrapeando la pag 1
estas escrapeando la pag 2
estas escrapeando la pag 3
estas escrapeando la pag 4
estas escrapeando la pag 5
estas escrapeando la pag 6
estas escrapeando la pag 7
estas escrapeando la pag 8
estas escrapeando la pag 9
estas escrapeando la pag 10
estas escrapeando la pag 11
estas escrapeando la pag 12
estas escrapeando la pag 13
estas escrapeando la pag 14
estas escrapeando la pag 15
estas escrapeando la pag 16


In [30]:
df_catalogo=pd.DataFrame(datos_book)
df_catalogo

,Titulo Revista,Area,Tematica,Titulo Articulo,Resumen,Abstract,Link Articulo,Link PDF
0,Revista de Investigación Clínica,III. Medicina y Ciencias de la Salud,Investigación Clínica,Quimioterapia Viral,None,None,https://produccioncientificaluz.org/index.php/...,https://produccioncientificaluz.org/index.php/...
1,Revista de Investigación Clínica,III. Medicina y Ciencias de la Salud,Investigación Clínica,La Angustia en la Alucinosis Alcohólica. Su Tr...,None,None,https://produccioncientificaluz.org/index.php/...,https://produccioncientificaluz.org/index.php/...
2,Revista de Investigación Clínica,III. Medicina y Ciencias de la Salud,Investigación Clínica,"""Anomalía de Pelger-Huet""",None,None,https://produccioncientificaluz.org/index.php/...,https://produccioncientificaluz.org/index.php/...
3,Revista de Investigación Clínica,III. Medicina y Ciencias de la Salud,Investigación Clínica,"""Un Caso de Púrpura Trombocitopenica Sintomáti...",None,None,https://produccioncientificaluz.org/index.php/...,https://produccioncientificaluz.org/index.php/...
4,Revista de Investigación Clínica,III. Medicina y Ciencias de la Salud,Investigación Clínica,Prevención y Tratamiento de la Gangrena Diabética,None,None,https://produccioncientificaluz.org/index.php/...,https://produccioncientificaluz.org/index.php/...
5,Revista de Investigación Clínica,III. Medicina y Ciencias de la Salud,Investigación Clínica,"""Tratamiento del Hipertiroidismo""",None,None,https://produccioncientificaluz.org/index.php/...,https://produccioncientificaluz.org/index.php/...
6,Revista de Investigación Clínica,III. Medicina y Ciencias de la Salud,Investigación Clínica,Hipotiroidismo en Trastornos Funcionales y Rep...,None,None,https://produccioncientificaluz.org/index.php/...,https://produccioncientificaluz.org/index.php/...
7,Revista de Investigación Clínica,III. Medicina y Ciencias de la Salud,Investigación Clínica,El Trabajo Submarino. Datos Históricos. Equipo...,None,None,https://produccioncientificaluz.org/index.php/...,https://produccioncientificaluz.org/index.php/...
8,Revista de Investigación Clínica,III. Medicina y Ciencias de la Salud,Investigación Clínica,Ventajas y Limitaciones de la Hipnosis en la P...,None,None,https://produccioncientificaluz.org/index.php/...,https://produccioncientificaluz.org/index.php/...
9,Revista de Investigación Clínica,III. Medicina y Ciencias de la Salud,Investigación Clínica,Algunos Aspectos de la Enfermedad Reumatoide,None,None,https://produccioncientificaluz.org/index.php/...,https://produccioncientificaluz.org/index.php/...


In [32]:
df_catalogo.to_csv('Revista035.02.csv', index=False)